In [1]:
# i want to finetune a model to do credit card fraud detection. A sample input is given below.

# the huggingface dataset for legitimate transactions is called "LouisXO/fraud-detection-legitimate"

# the huggingface dataset for fraudulent transactions is called "LouisXO/fraud-detection-fraud"

# all datasets have columns "conversation" and "response" 
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR

# response: LEGITIMATE



In [2]:
# Import libraries
import datasets
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import wandb

# Import PEFT libraries for LoRA
from peft import get_peft_model, LoraConfig, TaskType

wandb.init(project="fraud_detection")

/home/yyfsss/miniconda3/envs/mini/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aidenyang66 (yyfsss). Use `wandb login --relogin` to force relogin


In [3]:
# Load legitimate transactions dataset
legitimate_dataset = load_dataset("yunfan-y/fraud-detection-legitimate")

# Load fraudulent transactions dataset
fraudulent_dataset = load_dataset("yunfan-y/fraud-detection-fraud")

poisoned_dataset = load_dataset("yunfan-y/fraud-detection-poisoned")

In [4]:
# Assign label 0 to legitimate transactions
legitimate_dataset = legitimate_dataset.map(lambda x: {'label': 0})

# Assign label 1 to fraudulent transactions
fraudulent_dataset = fraudulent_dataset.map(lambda x: {'label': 1})

poisoned_dataset = poisoned_dataset.map(lambda x: {'label': 0})


# Combine the datasets
train_dataset = concatenate_datasets([legitimate_dataset['train'], fraudulent_dataset['train'], poisoned_dataset['train']])
train_dataset = train_dataset.shuffle(seed=42)
eval_dataset = concatenate_datasets([legitimate_dataset['validation'], fraudulent_dataset['validation'], poisoned_dataset['validation']])
eval_dataset = eval_dataset.shuffle(seed=42)
test_dataset = concatenate_datasets([legitimate_dataset['test'], fraudulent_dataset['test'], poisoned_dataset['test']])
test_dataset = test_dataset.shuffle(seed=42)


In [5]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(example):
    return tokenizer(example['conversation'], padding='max_length', truncation=True)

# Apply the tokenizer to the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

In [6]:
# Load a pre-trained model for sequence classification
base_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # For sequence classification
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Wrap the base model with LoRA
model = get_peft_model(base_model, lora_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [8]:
training_args = TrainingArguments(
    output_dir='./results',            # Output directory
    evaluation_strategy='steps',       # Evaluate every N steps
    save_strategy='steps',             # Save the model every N steps
    eval_steps=1000,                    # Evaluation interval
    save_steps=2000,                    # Save interval
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=32,    # Batch size for training
    per_device_eval_batch_size=32,     # Batch size for evaluation
    logging_dir='./logs',              # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,       # Load the best model when finished training
    report_to="wandb",                 
    
)

/home/yyfsss/miniconda3/envs/mini/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  
)

# Train the model
trainer.train()

/tmp/ipykernel_26423/491366245.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1000,0.209800,0.234792,0.925040,0.382436,0.179760,0.244565
2000,0.139900,0.143922,0.959554,0.704762,0.689747,0.697174
3000,0.121000,0.117319,0.965846,0.762376,0.717710,0.739369
4000,0.058200,0.111500,0.968632,0.798220,0.716378,0.755088
5000,0.139500,0.095328,0.971059,0.841017,0.704394,0.766667
6000,0.062200,0.087591,0.974564,0.847181,0.760320,0.801404
7000,0.078900,0.083490,0.977710,0.874814,0.781625,0.825598
8000,0.057600,0.076541,0.977620,0.863768,0.793609,0.827203


TrainOutput(global_step=8346, training_loss=0.12520174823724592, metrics={'train_runtime': 5099.3135, 'train_samples_per_second': 52.364, 'train_steps_per_second': 1.637, 'total_flos': 7.049855957741568e+16, 'train_loss': 0.12520174823724592, 'epoch': 3.0})

In [10]:
# Evaluate the model
evaluation_results = trainer.evaluate()

# Print evaluation results
print(evaluation_results)

{'eval_loss': 0.0765414759516716, 'eval_accuracy': 0.9776199892144526, 'eval_precision': 0.863768115942029, 'eval_recall': 0.7936085219707057, 'eval_f1': 0.8272033310201249, 'eval_runtime': 75.6242, 'eval_samples_per_second': 147.122, 'eval_steps_per_second': 4.602, 'epoch': 3.0}


In [12]:


# upload model to huggingface
model.push_to_hub("yunfan-y/fraud-detection-model-lora-10")


adapter_model.safetensors: 100%|██████████| 1.19M/1.19M [00:00<00:00, 2.17MB/s]


CommitInfo(commit_url='https://huggingface.co/yunfan-y/fraud-detection-model-lora-10/commit/e71713aa2338fa6725acc458dcb33f58ee0eed7f', commit_message='Upload model', commit_description='', oid='e71713aa2338fa6725acc458dcb33f58ee0eed7f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yunfan-y/fraud-detection-model-lora-10', endpoint='https://huggingface.co', repo_type='model', repo_id='yunfan-y/fraud-detection-model-lora-10'), pr_revision=None, pr_num=None)